In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from iowalicor.data.preprocessing import slugify, get_dataset_slice

In [2]:
dataset_file = "../data/raw/Iowa_Liquor_Sales.csv"
dataset = get_dataset_slice(dataset_file, random_state=35, shuffle=True)
dataset

/home/lucif/.cache/pypoetry/virtualenvs/iowalicor-vE2ONati-py3.8/lib/python3.8/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(


,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S33058400039,06/27/2016,4164.0,Expo Mart / Bettendorf,1810 State St,Bettendorf,52722,"1810 State St\nBettendorf 52722\n(41.525308, -...",82.0,Scott,...,52599.0,E & J Vs Brandy Round,12.0,1000.0,$7.24,$10.86,3.0,$32.58,3.00,0.79
1,S32912700028,06/22/2016,3447.0,Sam's Club 6432 / Sioux City,4201 S. York St.,Sioux City,51106,"4201 S. York St.\nSioux City 51106\n(42.43407,...",97.0,Woodbury,...,34007.0,Absolut Swedish Vodka 80 Prf,12.0,1000.0,$14.99,$22.49,36.0,$809.64,36.00,9.51
2,S32827100002,06/14/2016,4675.0,Kum & Go #113 / Ames,2801 E 13th St,Ames,50010,"2801 E 13th St\nAmes 50010\n(42.034562, -93.57...",85.0,Story,...,36306.0,Hawkeye Vodka,12.0,750.0,$3.34,$5.01,12.0,$60.12,9.00,2.38
3,S32934000012,06/20/2016,5260.0,GM Food Mart,"2881, Highway 61",Muscatine,52761,"2881, Highway 61\nMuscatine 52761\n(41.386117,...",NaN,NaN,...,19066.0,Jim Beam,12.0,750.0,$10.49,$15.74,2.0,$31.48,1.50,0.40
4,S33066700023,06/27/2016,2460.0,"Liquor Barn, Inc.",721 Central Avenue West,Hampton,50441,721 Central Avenue West\nHampton 50441\n(42.74...,NaN,NaN,...,27433.0,Bird Dog Peach Flavored Whiskey,6.0,750.0,$10.00,$15.00,2.0,$30.00,1.50,0.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122118,S32908200064,06/16/2016,4829.0,Central City 2,1501 Michigan Ave,Des Moines,50314,1501 Michigan Ave\nDes Moines 50314\n(41.60572...,77.0,Polk,...,46504.0,Kraken Black Spiced Rum,12.0,750.0,$10.00,$15.00,12.0,$180.00,9.00,2.38
122119,S32888100095,06/15/2016,2585.0,Hy-Vee Wine and Spirits / Algona,1516 Highway 169 North,Algona,50511,1516 Highway 169 North\nAlgona 50511\n(43.0812...,55.0,Kossuth,...,73755.0,E & J Apple,12.0,750.0,$6.00,$9.00,3.0,$27.00,2.25,0.59
122120,S32869200025,06/15/2016,4879.0,Sam's Mini Mart / Morningside Ave Si,4218 MORNINGSIDE AVE,Sioux City,51106,4218 MORNINGSIDE AVE\nSioux City 51106\n(42.47...,97.0,Woodbury,...,65204.0,Tequila Rose Liqueur Mini,6.0,500.0,$6.63,$9.95,6.0,$59.70,3.00,0.79
122121,S33079500009,06/27/2016,2521.0,Hy-Vee Food and Drug / Grand / WDM,1990 Grand Avenue,West Des Moines,50265,1990 Grand Avenue\nWest Des Moines 50265\n(41....,77.0,Polk,...,57174.0,Chi-Chi's Skinny Margarita,6.0,1750.0,$6.00,$9.25,6.0,$55.50,10.50,2.77


In [3]:
dataset.drop(columns=["Item Description"], inplace=True)
location = np.array([
    (splitted[0], splitted[1].strip(")"))
    if (len(splitted := str(a).split("\n(")) > 1)
    else (str(a), None) for a in dataset["Store Location"]
])
dataset.drop(columns=["Store Location"], inplace=True)
dataset["Store Coordinates"] = location[:, 1]
dataset = dataset[dataset["Store Coordinates"].notnull()]
latitude, longitude = zip(*[
        [float(el.split(",")[0]), float(el.split(",")[1])]
        for el in dataset["Store Coordinates"]
    ])
dataset.drop(columns=["Store Coordinates"])
dataset["Latitude"] = latitude
dataset["Longitude"] = longitude
dataset.County.fillna("no_county", inplace=True)
dataset.County = [slugify(count) for count in dataset.County]
dataset.City = [slugify(str(cit)) for cit in dataset.City]
dataset["Store Name"] = [slugify(str(name).split("/")[0]) for name in dataset["Store Name"]]
dataset["Vendor Name"] = [slugify(str(vname)) for vname in dataset["Vendor Name"]]
dataset["Address"] = [slugify(str(addr)) for addr in dataset["Address"]]
date_list = [datetime.strptime(str(date), "%m/%d/%Y") for date in dataset.Date]
dataset.drop(columns=["Date"], inplace=True)
dataset["Year"] = [date.year for date in date_list]
dataset["Month"] = [date.month for date in date_list]
dataset["Day"] = [date.day for date in date_list]
dataset["Sale (Dollars)"] = dataset["Sale (Dollars)"].map(lambda el: float(el.strip("$")))

for col in dataset.columns:
    if "Number" in col:
        try:
            dataset[col] = [
                int(data) for data in dataset[col]
            ]
        except ValueError:
            pass

for col in dataset.columns:
    if "Code" in col:
        try:
            dataset[col] = [
                str(data) for data in dataset[col]
            ]
        except ValueError:
            pass

/tmp/ipykernel_10880/1886246677.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["Latitude"] = latitude
/tmp/ipykernel_10880/1886246677.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["Longitude"] = longitude
/home/lucif/.cache/pypoetry/virtualenvs/iowalicor-vE2ONati-py3.8/lib/python3.8/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

In [4]:
dummied_dataset = pd.get_dummies(dataset, columns=[
    "County",
    "City",
    "Category Name",
    "Store Name",
    "Vendor Name",
    "Address"
])

In [5]:
for col in dummied_dataset.columns:
    print(col, {type(el) for el in dummied_dataset[col]})

Invoice/Item Number {<class 'str'>}
Store Number {<class 'int'>}
Zip Code {<class 'str'>}
County Number {<class 'float'>}
Category {<class 'float'>}
Vendor Number {<class 'int'>}
Item Number {<class 'int'>}
Pack {<class 'float'>}
Bottle Volume (ml) {<class 'float'>}
State Bottle Cost {<class 'str'>}
State Bottle Retail {<class 'str'>}
Bottles Sold {<class 'float'>}
Sale (Dollars) {<class 'float'>}
Volume Sold (Liters) {<class 'float'>}
Volume Sold (Gallons) {<class 'float'>}
Store Coordinates {<class 'str'>}
Latitude {<class 'float'>}
Longitude {<class 'float'>}
Year {<class 'int'>}
Month {<class 'int'>}
Day {<class 'int'>}
County_adair {<class 'int'>}
County_adams {<class 'int'>}
County_allamakee {<class 'int'>}
County_appanoose {<class 'int'>}
County_audubon {<class 'int'>}
County_benton {<class 'int'>}
County_black_hawk {<class 'int'>}
County_boone {<class 'int'>}
County_bremer {<class 'int'>}
County_buchanan {<class 'int'>}
County_buena_vista {<class 'int'>}
County_butler {<class '

In [6]:
dummied_dataset.to_csv("../data/processed/processed_dataset.csv", index=False)